In [ ]:
import cv2
import time
import numpy as np
import tensorflow as tf  # Import TensorFlow để sử dụng mô hình đã huấn luyện (thay vì tflite_runtime)
from collections import Counter

# Tải mô hình phát hiện khuôn mặt (SSD) sử dụng thư viện OpenCV DNN
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

# Tải mô hình TensorFlow đã huấn luyện (mô hình EfficientNetB0)
model = tf.keras.models.load_model("efficientnetb0.h5")  # Tải mô hình phân loại mặt có khẩu trang hoặc không

# Các nhãn phân loại (các lớp trong mô hình nhận diện khẩu trang)
labels = ["incorrect_mask", "with_mask", "without_mask"]

# Màu sắc cho mỗi nhãn, sử dụng để vẽ bounding box với màu sắc tương ứng
label_colors = {
    "incorrect_mask": (0, 165, 255),   # Màu vàng cho người đeo khẩu trang sai
    "with_mask": (0, 255, 0),          # Màu xanh lá cho người đeo khẩu trang đúng
    "without_mask": (0, 0, 255)        # Màu đỏ cho người không đeo khẩu trang
}

# Mở webcam (ID của webcam có thể thay đổi, thông thường là 0 hoặc 1)
cap = cv2.VideoCapture(1)  # Mở webcam thứ 2 nếu có (có thể thử thay đổi thành 0 nếu không hoạt động)
frame_count = 0  # Đếm số khung hình đã xử lý
detect_interval = 10  # Chỉ phát hiện khuôn mặt mỗi 10 khung hình để giảm tải
results = []  # Mảng lưu kết quả (nhãn và độ tin cậy)
boxes = []  # Mảng lưu thông tin bounding box (vị trí khuôn mặt)

start_time = time.perf_counter()  # Thời gian bắt đầu để tính FPS
fps = 0  # Khởi tạo biến FPS

while True:
    ret, frame = cap.read()  # Đọc từng khung hình từ webcam
    if not ret:  # Kiểm tra nếu không đọc được khung hình
        break  # Thoát nếu không đọc được

    frame_count += 1  # Tăng số lượng khung hình đã xử lý
    h, w = frame.shape[:2]  # Lấy chiều cao (h) và chiều rộng (w) của khung hình

    # Chỉ thực hiện phát hiện mặt mỗi 10 khung hình (theo detect_interval)
    if frame_count % detect_interval < 3:
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                     (104.0, 177.0, 123.0))  # Chuẩn hóa ảnh đầu vào trước khi đưa vào mô hình SSD
        net.setInput(blob)  # Cung cấp ảnh đã chuẩn hóa vào mô hình SSD
        detections = net.forward()  # Phát hiện các khuôn mặt trong ảnh
        results = []  # Khởi tạo lại danh sách kết quả cho mỗi khung hình
        boxes = []  # Khởi tạo lại danh sách bounding box

        # Duyệt qua tất cả các phát hiện mặt trong ảnh
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]  # Lấy độ tin cậy (confidence) của phát hiện khuôn mặt
            if confidence > 0.5:  # Chỉ xử lý nếu độ tin cậy > 50%
                # Tính toán vị trí của bounding box trong ảnh
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])  # Chuyển đổi tỷ lệ với kích thước ảnh gốc
                x1, y1, x2, y2 = box.astype("int")  # Lấy toạ độ của bounding box
                x1, y1 = max(0, x1), max(0, y1)  # Đảm bảo toạ độ không vượt quá biên giới ảnh
                x2, y2 = min(w - 1, x2), min(h - 1, y2)
                face_img = frame[y1:y2, x1:x2]  # Cắt phần khuôn mặt ra từ ảnh

                if face_img.size == 0:  # Nếu không tìm thấy khuôn mặt (kích thước ảnh = 0), bỏ qua
                    continue

                # Resize khuôn mặt về kích thước 224x224 và chuẩn hóa giá trị
                face_resized = cv2.resize(face_img, (224, 224))  # Resize ảnh mặt về kích thước 224x224
                face_normalized = face_resized.astype("float32") / 255.0  # Chuẩn hóa giá trị pixel (từ 0-255 sang 0-1)
                face_array = np.expand_dims(face_normalized, axis=0)  # Thêm một chiều để chuẩn bị dữ liệu cho mô hình

                # Dự đoán nhãn từ mô hình TensorFlow
                predictions = model.predict(face_array)  # Dự đoán nhãn cho khuôn mặt
                class_index = int(np.argmax(predictions))  # Lấy chỉ số lớp có xác suất cao nhất
                conf = predictions[0][class_index]  # Lấy độ tin cậy của lớp đó
                label = labels[class_index]  # Lấy tên nhãn tương ứng với chỉ số lớp

                results.append((label, conf))  # Lưu kết quả dự đoán vào danh sách
                boxes.append((x1, y1, x2, y2, label, conf))  # Lưu thông tin bounding box và nhãn vào danh sách

    # Vẽ bounding box và nhãn lên ảnh
    for (x1, y1, x2, y2, label, conf) in boxes:
        color = label_colors.get(label, (255, 255, 255))  # Chọn màu sắc tương ứng với nhãn
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Vẽ bounding box quanh khuôn mặt
        text = f"{label} ({conf*100:.1f}%)"  # Tạo chuỗi văn bản để hiển thị nhãn và độ tin cậy
        cv2.putText(frame, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)  # Hiển thị văn bản lên ảnh

    # Tính FPS mỗi 10 khung hình để theo dõi tốc độ xử lý
    if frame_count % 10 == 0:
        end_time = time.perf_counter()  # Lấy thời gian kết thúc sau khi xử lý 10 khung hình
        fps = 10 / (end_time - start_time)  # Tính số khung hình mỗi giây (FPS)
        start_time = end_time  # Cập nhật lại thời gian bắt đầu

    # Hiển thị FPS lên video (tốc độ khung hình)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Face Mask Detection", frame)  # Hiển thị khung hình video với kết quả
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Thoát nếu nhấn phím 'q'
        break

cap.release()  # Giải phóng tài nguyên webcam khi kết thúc
cv2.destroyAllWindows()  # Đóng tất cả cửa sổ OpenCV